In [101]:
# Importation des librairies
import streamlit as st
from pandas_datareader.data import DataReader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import date
import plotly.express as px
import plotly.tools as tls
import yfinance as yf

# URLs pour les indices boursiers
url_sp500 = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"  # USA
url_cac40 = "https://en.wikipedia.org/wiki/CAC_40"  # Paris
url_ftse100 = "https://en.wikipedia.org/wiki/FTSE_100_Index"  # Londres
url_nikkei = "https://bullishbears.com/nikkei-225-stocks-list/#:~:text=The%20Bullish%20Bears%20break%20down%20a%20list%20of%20the%20Nikkei"  # Tokyo
url_dax = "https://en.wikipedia.org/wiki/DAX"  # Berlin

# Fonction pour lire les tables en toute sécurité
def read_table(url, index):
    try:
        df = pd.read_html(url)[index]
        st.success(f"Table lue avec succès depuis {url}.")
        return df
    except Exception as e:
        st.error(f"Erreur lors de la lecture de la table à partir de {url} : {e}")
        return pd.DataFrame()  # Retourne un DataFrame vide en cas d'erreur

# Chargement et traitement des données pour chaque indice boursier
dax = read_table(url_dax, 4)
if 'Company' in dax.columns and 'Ticker' in dax.columns:
    dax['NameOfStock'] = dax['Company'] + "_" + dax['Ticker']

nikkei = read_table(url_nikkei, 0)
if 'Name' in nikkei.columns and 'Symbol' in nikkei.columns:
    nikkei['Name'] = nikkei['Name'].replace(",", "", regex=True)  # Supprimer les virgules
    nikkei['NameOfStock'] = nikkei['Name'] + "_" + nikkei['Symbol'] + ".T"

sp500 = read_table(url_sp500, 0)
if 'Security' in sp500.columns and 'Symbol' in sp500.columns:
    sp500['NameOfStock'] = sp500['Security'] + "_" + sp500['Symbol']

cac40 = read_table(url_cac40, 4)
if 'Company' in cac40.columns and 'Ticker' in cac40.columns:
    cac40['NameOfStock'] = cac40['Company'] + "_" + cac40['Ticker']

ftse100 = read_table(url_ftse100, 4)
if 'Company' in ftse100.columns and 'Ticker' in ftse100.columns:
    ftse100['NameOfStock'] = ftse100['Company'] + "_" + ftse100["Ticker"]

In [102]:
nikkei

,Symbol,Name,Market Cap,Sector,Industry,NameOfStock
0,6758,Sony Group Corporation,$126.64 B,Technology,Consumer Electronics,Sony Group Corporation_6758.T
1,6861,Keyence Corp,$113.20 B,Technology,Scientific & Technical Instruments,Keyence Corp_6861.T
2,9432,Nippon Telegraph & Telephone Corp,$103.75 B,Communication Services,Telecom Services,Nippon Telegraph & Telephone Corp_9432.T
3,8306,Mitsubishi UFJ Financial Group Inc.,$85.03 B,Financials,Banks,Mitsubishi UFJ Financial Group Inc._8306.T
4,8035,Tokyo Electron Ltd,$79.62 B,Technology,Semiconductor Equipment & Materials,Tokyo Electron Ltd_8035.T
...,...,...,...,...,...,...
219,4042,Tosoh Corp,$4.92 B,Basic Materials,Chemicals,Tosoh Corp_4042.T
220,3861,Oji Holdings Corp,$4.91 B,Basic Materials,Paper & Paper Products,Oji Holdings Corp_3861.T
221,7261,Mazda Motor Corp,$4.83 B,Consumer Discretionary,Auto Manufacturers,Mazda Motor Corp_7261.T
222,9008,Keio Corporation,$4.83 B,Industrials,Railroads,Keio Corporation_9008.T
